# 🗺️ Análise Espacial e Mapas Interativos

Este notebook demonstra:
- Manipulação de dados geoespaciais com GeoPandas
- Criação de mapas estáticos (matplotlib)
- Mapas interativos (folium)
- Análise de proximidade e buffer
- Análise de densidade espacial
- Design minimalista em preto e branco

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point, Polygon
from shapely import wkt
import folium
from folium import plugins
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Configuração de estilo minimalista
plt.style.use('grayscale')
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10

print("✓ Bibliotecas importadas com sucesso")

## 1. Carregar Dados e Criar Geometrias

In [ ]:
# Carregar dados processados ou criar exemplos
try:
    df_lotes = pd.read_parquet('../data/processed/lotes_processados.parquet')
    print(f"✓ Dados de lotes carregados: {len(df_lotes)} registros")
except FileNotFoundError:
    print("⚠️ Criando dados de exemplo com geometrias...")
    
    # Coordenadas aproximadas de Vitória, ES
    vitoria_center = [-20.3155, -40.3128]
    
    # Criar dados de exemplo com coordenadas
    np.random.seed(42)
    n_lotes = 150
    
    # Gerar pontos aleatórios ao redor de Vitória
    lats = np.random.normal(vitoria_center[0], 0.02, n_lotes)
    lons = np.random.normal(vitoria_center[1], 0.02, n_lotes)
    
    df_lotes = pd.DataFrame({
        'codLote': range(1, n_lotes + 1),
        'logradouro': [f'Rua {i%20}' for i in range(n_lotes)],
        'numero': np.random.randint(1, 1000, n_lotes),
        'bairro': np.random.choice(['Centro', 'Praia do Canto', 'Jardim Camburi', 'Enseada do Suá', 'Bento Ferreira'], n_lotes),
        'sigla_trat': np.random.choice(['R1', 'R2', 'COM', 'MISTO'], n_lotes),
        'area_terreno': np.random.uniform(200, 2000, n_lotes),
        'ca': np.random.uniform(1.0, 4.0, n_lotes),
        'to': np.random.uniform(0.3, 0.8, n_lotes),
        'limite_altura': np.random.choice([12, 18, 24, 30, 50], n_lotes),
        'altura': np.random.uniform(12, 60, n_lotes),
        'latitude': lats,
        'longitude': lons
    })
    
    print(f"✓ Dados de exemplo criados: {len(df_lotes)} lotes")

# Se não tiver geometry, criar a partir de lat/lon
if 'geometry' not in df_lotes.columns:
    if 'latitude' in df_lotes.columns and 'longitude' in df_lotes.columns:
        geometry = [Point(xy) for xy in zip(df_lotes['longitude'], df_lotes['latitude'])]
        gdf_lotes = gpd.GeoDataFrame(df_lotes, geometry=geometry, crs='EPSG:4326')
    else:
        # Criar geometrias de exemplo
        vitoria_center = [-20.3155, -40.3128]
        lats = np.random.normal(vitoria_center[0], 0.02, len(df_lotes))
        lons = np.random.normal(vitoria_center[1], 0.02, len(df_lotes))
        geometry = [Point(xy) for xy in zip(lons, lats)]
        gdf_lotes = gpd.GeoDataFrame(df_lotes, geometry=geometry, crs='EPSG:4326')
else:
    # Converter coluna geometry de string WKT para geometria
    if df_lotes['geometry'].dtype == 'object':
        df_lotes['geometry'] = df_lotes['geometry'].apply(wkt.loads)
    gdf_lotes = gpd.GeoDataFrame(df_lotes, geometry='geometry', crs='EPSG:4326')

print(f"\n✓ GeoDataFrame criado com {len(gdf_lotes)} geometrias")
print(f"CRS: {gdf_lotes.crs}")
print(f"\nBounds: {gdf_lotes.total_bounds}")

gdf_lotes.head()

## 2. Mapa Estático - Distribuição dos Lotes

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 12))

# Plot base
gdf_lotes.plot(ax=ax, 
               marker='o', 
               color='black', 
               markersize=50,
               alpha=0.6,
               edgecolor='white',
               linewidth=0.5)

ax.set_title('Distribuição Espacial dos Lotes em Vitória', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_facecolor('white')

# Adicionar anotações
ax.text(0.02, 0.98, f'Total de Lotes: {len(gdf_lotes):,}', 
        transform=ax.transAxes, 
        fontsize=12, 
        verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='black'))

plt.tight_layout()
plt.show()

print("✓ Mapa estático gerado")

## 3. Análise por Bairro - Mapa Colorido por Zoneamento

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Mapa 1: Colorido por Bairro
bairros_unicos = gdf_lotes['bairro'].unique()
colors_bairro = {bairro: i for i, bairro in enumerate(bairros_unicos)}
gdf_lotes['color_bairro'] = gdf_lotes['bairro'].map(colors_bairro)

gdf_lotes.plot(ax=axes[0], 
               column='bairro',
               categorical=True,
               legend=True,
               markersize=100,
               alpha=0.7,
               edgecolor='black',
               linewidth=0.5,
               cmap='gray')

axes[0].set_title('Distribuição por Bairro', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].grid(True, alpha=0.3)

# Mapa 2: Colorido por Zoneamento
gdf_lotes.plot(ax=axes[1], 
               column='sigla_trat',
               categorical=True,
               legend=True,
               markersize=100,
               alpha=0.7,
               edgecolor='black',
               linewidth=0.5,
               cmap='gray')

axes[1].set_title('Distribuição por Zoneamento', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Mapas de distribuição gerados")

## 4. Mapa de Calor - Área dos Terrenos

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Mapa 1: Área do Terreno
gdf_lotes.plot(ax=axes[0], 
               column='area_terreno',
               cmap='Greys',
               legend=True,
               markersize=150,
               alpha=0.7,
               edgecolor='black',
               linewidth=0.5,
               legend_kwds={'label': 'Área do Terreno (m²)', 'orientation': 'horizontal'})

axes[0].set_title('Mapa de Calor: Área do Terreno', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].grid(True, alpha=0.3)

# Mapa 2: Coeficiente de Aproveitamento
gdf_lotes.plot(ax=axes[1], 
               column='ca',
               cmap='Greys',
               legend=True,
               markersize=150,
               alpha=0.7,
               edgecolor='black',
               linewidth=0.5,
               legend_kwds={'label': 'Coeficiente de Aproveitamento', 'orientation': 'horizontal'})

axes[1].set_title('Mapa de Calor: Coeficiente de Aproveitamento (CA)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Mapas de calor gerados")

## 5. Mapa Interativo com Folium - Básico

In [ ]:
# Calcular centro do mapa
center_lat = gdf_lotes.geometry.y.mean()
center_lon = gdf_lotes.geometry.x.mean()

# Criar mapa base (monocromático)
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='CartoDB positron',  # Estilo minimalista claro
    control_scale=True
)

# Adicionar marcadores para cada lote
for idx, row in gdf_lotes.iterrows():
    # Criar popup com informações
    popup_html = f"""
    <div style="font-family: Arial; font-size: 12px; color: black;">
        <b>Lote: {row['codLote']}</b><br>
        <hr style="margin: 5px 0; border: 1px solid #ccc;">
        📍 {row['logradouro']}, {row['numero']}<br>
        🏘️ Bairro: {row['bairro']}<br>
        📏 Área: {row['area_terreno']:.2f} m²<br>
        🏗️ CA: {row['ca']:.2f}<br>
        📐 TO: {row['to']:.2%}<br>
        📊 Altura: {row['altura']:.2f} m<br>
        🗂️ Zona: {row['sigla_trat']}
    </div>
    """
    
    # Definir cor baseada no zoneamento
    color_map = {
        'R1': 'lightgray',
        'R2': 'gray',
        'COM': 'black',
        'MISTO': 'darkgray'
    }
    
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        popup=folium.Popup(popup_html, max_width=300),
        color='black',
        fillColor=color_map.get(row['sigla_trat'], 'gray'),
        fillOpacity=0.7,
        weight=1
    ).add_to(m)

# Adicionar legenda
legend_html = '''
<div style="position: fixed; 
            top: 10px; right: 10px; width: 180px; height: 140px; 
            background-color: white; border: 2px solid black;
            z-index: 9999; font-size: 12px; padding: 10px;">
    <b style="font-size: 14px;">Zoneamento</b><br>
    <i class="fa fa-circle" style="color:lightgray"></i> R1 - Residencial 1<br>
    <i class="fa fa-circle" style="color:gray"></i> R2 - Residencial 2<br>
    <i class="fa fa-circle" style="color:black"></i> COM - Comercial<br>
    <i class="fa fa-circle" style="color:darkgray"></i> MISTO - Uso Misto
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Adicionar layer de tela cheia
plugins.Fullscreen().add_to(m)

# Salvar mapa
m.save('../data/processed/mapa_lotes_interativo.html')

print("✓ Mapa interativo criado: mapa_lotes_interativo.html")
print(f"  • Centro: [{center_lat:.4f}, {center_lon:.4f}]")
print(f"  • {len(gdf_lotes)} marcadores adicionados")

# Exibir mapa
m

## 6. Análise de Buffer e Proximidade

In [ ]:
# Converter para sistema de coordenadas métrico (UTM)
gdf_lotes_utm = gdf_lotes.to_crs(epsg=32724)  # UTM Zone 24S para Vitória

# Selecionar um ponto de interesse (primeiro lote como exemplo)
ponto_interesse = gdf_lotes_utm.iloc[0].geometry

# Criar buffers de 500m, 1000m e 1500m
buffer_500 = ponto_interesse.buffer(500)
buffer_1000 = ponto_interesse.buffer(1000)
buffer_1500 = ponto_interesse.buffer(1500)

# Contar lotes dentro de cada buffer
lotes_500 = gdf_lotes_utm[gdf_lotes_utm.geometry.within(buffer_500)]
lotes_1000 = gdf_lotes_utm[gdf_lotes_utm.geometry.within(buffer_1000)]
lotes_1500 = gdf_lotes_utm[gdf_lotes_utm.geometry.within(buffer_1500)]

# Visualizar
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# Plot buffers
gpd.GeoSeries([buffer_1500]).plot(ax=ax, color='lightgray', alpha=0.3, edgecolor='black', linewidth=2, label='1500m')
gpd.GeoSeries([buffer_1000]).plot(ax=ax, color='gray', alpha=0.4, edgecolor='black', linewidth=2, label='1000m')
gpd.GeoSeries([buffer_500]).plot(ax=ax, color='darkgray', alpha=0.5, edgecolor='black', linewidth=2, label='500m')

# Plot lotes
gdf_lotes_utm.plot(ax=ax, color='black', markersize=30, alpha=0.6, edgecolor='white', linewidth=0.5)

# Plot ponto de interesse
gpd.GeoSeries([ponto_interesse]).plot(ax=ax, color='white', markersize=200, edgecolor='black', linewidth=3, marker='*', label='Ponto de Interesse')

ax.set_title('Análise de Proximidade - Buffers ao Redor do Ponto de Interesse', fontsize=14, fontweight='bold')
ax.set_xlabel('UTM Este (m)')
ax.set_ylabel('UTM Norte (m)')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_facecolor('white')

plt.tight_layout()
plt.show()

print("\n📍 ANÁLISE DE PROXIMIDADE")
print("="*50)
print(f"Raio de 500m:  {len(lotes_500):3d} lotes")
print(f"Raio de 1000m: {len(lotes_1000):3d} lotes")
print(f"Raio de 1500m: {len(lotes_1500):3d} lotes")
print("\nÁrea média dos lotes por raio:")
print(f"  • 500m:  {lotes_500['area_terreno'].mean():.2f} m²")
print(f"  • 1000m: {lotes_1000['area_terreno'].mean():.2f} m²")
print(f"  • 1500m: {lotes_1500['area_terreno'].mean():.2f} m²")

## 7. Densidade Espacial - KDE Plot

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Extrair coordenadas
x = gdf_lotes.geometry.x
y = gdf_lotes.geometry.y

# Plot 1: Densidade de pontos (scatter)
axes[0].scatter(x, y, c='black', alpha=0.5, s=50, edgecolors='white', linewidth=0.5)
axes[0].set_title('Distribuição dos Lotes', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].grid(True, alpha=0.3)
axes[0].set_facecolor('white')

# Plot 2: Densidade KDE
from scipy.stats import gaussian_kde

# Criar grid para KDE
xx, yy = np.mgrid[x.min():x.max():100j, y.min():y.max():100j]
positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

# Plot contorno
contour = axes[1].contourf(xx, yy, f, levels=15, cmap='Greys', alpha=0.8)
axes[1].scatter(x, y, c='black', alpha=0.3, s=20, edgecolors='white', linewidth=0.3)
axes[1].set_title('Densidade Espacial (KDE)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].grid(True, alpha=0.3)
plt.colorbar(contour, ax=axes[1], label='Densidade')

plt.tight_layout()
plt.show()

print("✓ Análise de densidade espacial concluída")

## 8. Mapa de Calor Interativo (Heatmap com Folium)

In [ ]:
# Criar mapa base
m_heat = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='CartoDB positron'
)

# Preparar dados para heatmap
heat_data = [[row.geometry.y, row.geometry.x] for idx, row in gdf_lotes.iterrows()]

# Adicionar camada de calor
plugins.HeatMap(
    heat_data,
    min_opacity=0.2,
    max_val=1.0,
    radius=15,
    blur=15,
    gradient={0.0: 'white', 0.5: 'gray', 1.0: 'black'}  # Gradiente preto e branco
).add_to(m_heat)

# Adicionar fullscreen
plugins.Fullscreen().add_to(m_heat)

# Salvar
m_heat.save('../data/processed/mapa_calor_lotes.html')

print("✓ Mapa de calor interativo criado: mapa_calor_lotes.html")

# Exibir
m_heat

## 9. Análise de Clusters Espaciais

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Preparar dados para clustering
coords = np.array([[pt.x, pt.y] for pt in gdf_lotes.geometry])

# Aplicar DBSCAN
db = DBSCAN(eps=0.01, min_samples=5)
clusters = db.fit_predict(coords)

# Adicionar cluster ao GeoDataFrame
gdf_lotes['cluster'] = clusters

# Contar clusters
n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
n_noise = list(clusters).count(-1)

print(f"\n🎯 ANÁLISE DE CLUSTERS")
print("="*50)
print(f"Número de clusters encontrados: {n_clusters}")
print(f"Número de pontos de ruído: {n_noise}")

# Visualizar clusters
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# Plot cada cluster com tonalidade diferente de cinza
unique_clusters = sorted(set(clusters))
colors = plt.cm.Greys(np.linspace(0.3, 0.9, len(unique_clusters)))

for cluster_id, color in zip(unique_clusters, colors):
    if cluster_id == -1:
        # Ruído em vermelho escuro (preto)
        cluster_data = gdf_lotes[gdf_lotes['cluster'] == cluster_id]
        cluster_data.plot(ax=ax, color='black', markersize=30, alpha=0.3, label='Ruído', edgecolor='white', linewidth=0.5)
    else:
        cluster_data = gdf_lotes[gdf_lotes['cluster'] == cluster_id]
        cluster_data.plot(ax=ax, color=color, markersize=80, alpha=0.7, label=f'Cluster {cluster_id}', edgecolor='black', linewidth=0.5)

ax.set_title('Clusters Espaciais de Lotes (DBSCAN)', fontsize=14, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend(loc='upper right', fontsize=9, ncol=2)
ax.grid(True, alpha=0.3)
ax.set_facecolor('white')

plt.tight_layout()
plt.show()

# Estatísticas por cluster
print("\n📊 Estatísticas por Cluster:")
cluster_stats = gdf_lotes.groupby('cluster').agg({
    'codLote': 'count',
    'area_terreno': 'mean',
    'ca': 'mean',
    'to': 'mean'
}).round(2)
cluster_stats.columns = ['Qtd Lotes', 'Área Média (m²)', 'CA Médio', 'TO Médio']
print(cluster_stats)

## 10. Exportar GeoDataFrame com Análises

In [ ]:
# Salvar GeoDataFrame com clusters
gdf_lotes.to_file('../data/processed/lotes_com_clusters.geojson', driver='GeoJSON')
gdf_lotes.to_parquet('../data/processed/lotes_com_analise_espacial.parquet')

print("✓ Dados exportados:")
print("  • lotes_com_clusters.geojson")
print("  • lotes_com_analise_espacial.parquet")
print(f"  • {len(gdf_lotes)} registros com {len(gdf_lotes.columns)} colunas")
print(f"\nColunas incluídas: {list(gdf_lotes.columns)}")